In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import utils  # your utility module
import ssl
from pymongo import MongoClient
import os
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import CohereRerank


load_dotenv(override=True)




True

In [5]:
data = utils.read_data()
print(f"Data loaded: {len(data)} characters")


Data loaded: 45744 characters


In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = splitter.create_documents([data])
print(f"Number of chunks created: {len(docs)}")


Number of chunks created: 101


In [7]:
# Step 3: Create embeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')


/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_32849/2019018592.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model='text-embedding-3-small')


In [8]:

load_dotenv(override=True)


openai_api_key = os.environ.get("OPENAI_API_KEY")
google_api_key = os.environ.get("GOOGLE_API_KEY")



client = MongoClient(os.getenv("MONGODB_URI"))
collection = client["RAG-evaluation"]["RAG-simple"]


In [9]:

# Step 4: Create FAISS vector store
# vectorstore = MongoDBAtlasVectorSearch.from_documents(
#             documents=docs,
#             embedding=embedding,
#             collection=collection,
#             index_name="vector_index"
#         )
print("Vector store created successfully.")


Vector store created successfully.


In [ ]:
# Create retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

vectorstore = MongoDBAtlasVectorSearch(collection=collection, embedding=embedding, index_name="vector_index")

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})

# Wrap it with reranker
reranker = CohereRerank(
    model="rerank-english-v3.0",  # Cohere rerank model
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever
)


# Example query
query = "how to delay skin aging"
results = compression_retriever.get_relevant_documents(query)

for i, r in enumerate(results):
    print(f"Rank {i+1}: {r.page_content[:200]}")

/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_32849/1060519461.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = compression_retriever.get_relevant_documents(query)


UnauthorizedError: headers: {'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'date': 'Sun, 17 Aug 2025 21:57:04 GMT', 'content-length': '75', 'x-envoy-upstream-service-time': '1', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 401, body: {'id': '54c729b5-103b-4818-827e-a50855d4c44c', 'message': 'invalid api token'}